# Table of Contents

##### - [Imports and data overview](#overview)

### Imports and SparkSession creation

In [37]:
import pandas as pd 
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, col, when, count
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
import tensorflow as tf
import logging
import warnings

warnings.filterwarnings("ignore")

In [10]:
spark = SparkSession.builder.master("local").appName("health").getOrCreate()

In [40]:
spark.sparkContext.setLogLevel("ERROR")

### Get and initially view Dataset

In [59]:
df = spark.read.csv("health_data_static/gym_members_exercise_tracking.csv", inferSchema=True, header=True)
#Let's view head(3), to see what the dataset we have in question
df.show(n=3)
#Number of rows:
df.count()

+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
|Age|Gender|Weight (kg)|Height (m)|Max_BPM|Avg_BPM|Resting_BPM|Session_Duration (hours)|Calories_Burned|Workout_Type|Fat_Percentage|Water_Intake (liters)|Workout_Frequency (days/week)|Experience_Level|  BMI|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
| 56|  Male|       88.3|      1.71|    180|    157|         60|                    1.69|         1313.0|        Yoga|          12.6|                  3.5|                            4|               3| 30.2|
| 46|Female|       74.9|      1.53|    179|    151|         66|                     1.3|          883.0|        HIIT|          33.9|                  2.1|              

973

In [62]:
#Howerver, let's assume we're working with a dynamic stream. In this case, we choose to append, assuming new additions may be added
schema = StructType( [
    StructField("Age", IntegerType(), True),                StructField("Gender", StringType(), True),
    StructField("Weight_(kg)", FloatType(), True),          StructField("Height_(m)", FloatType(), True),
    StructField("Max_BPM", IntegerType(), True),            StructField("Avg_BPM", IntegerType(), True),
    StructField("Resting_BPM", IntegerType(), True),        StructField("Session_Duration_(hours)", FloatType(), True), 
    StructField("Calories_Burned", FloatType(), True),      StructField("Workout_Type", StringType(), True),
    StructField("Fat_Percentage", FloatType(), True),       StructField("Water_Intake_(liters)", FloatType(), True),
    StructField("Workout_Frequency_(days/week)", IntegerType(), True),
    StructField("Experience_Level", IntegerType(), True),   StructField("BMI", FloatType(), True)
])

df = spark.readStream.schema(schema).csv("health_data")

### Let's view by batch:

Although our streaming directory has two separate data files, the stream consolidates them into one.

In [63]:
def show_head_and_shape(batch_df, batch_id):
    print(f"batch {batch_id} has {batch_df.count()} rows")
    batch_df.show(n=1)
    print("\n")

query = df.writeStream.foreachBatch(show_head_and_shape).start()

batch 0 has 973 rows
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+----+
|Age|Gender|Weight_(kg)|Height_(m)|Max_BPM|Avg_BPM|Resting_BPM|Session_Duration_(hours)|Calories_Burned|Workout_Type|Fat_Percentage|Water_Intake_(liters)|Workout_Frequency_(days/week)|Experience_Level| BMI|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+----+
| 56|  Male|       88.3|      1.71|    180|    157|         60|                    1.69|         1313.0|        Yoga|          12.6|                  3.5|                            4|               3|30.2|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+-------------------

In [54]:
#Show first rows
query = df.writeStream.outputMode("append").format("console").start()

-------------------------------------------
Batch: 0
-------------------------------------------
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
|Age|Gender|Weight_(kg)|Height_(m)|Max_BPM|Avg_BPM|Resting_BPM|Session_Duration_(hours)|Calories_Burned|Workout_Type|Fat_Percentage|Water_Intake_(liters)|Workout_Frequency_(days/week)|Experience_Level|  BMI|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
| 56|  Male|       88.3|      1.71|    180|    157|         60|                    1.69|         1313.0|        Yoga|          12.6|                  3.5|                            4|               3| 30.2|
| 46|Female|       74.9|      1.53|    179|    151|         66|        

In [45]:
df.count()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
FileSource[health_data]

In [7]:
df.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Weight (kg): string (nullable = true)
 |-- Height (m): string (nullable = true)
 |-- Max_BPM: string (nullable = true)
 |-- Avg_BPM: string (nullable = true)
 |-- Resting_BPM: string (nullable = true)
 |-- Session_Duration (hours): string (nullable = true)
 |-- Calories_Burned: string (nullable = true)
 |-- Workout_Type: string (nullable = true)
 |-- Fat_Percentage: string (nullable = true)
 |-- Water_Intake (liters): string (nullable = true)
 |-- Workout_Frequency (days/week): string (nullable = true)
 |-- Experience_Level: string (nullable = true)
 |-- BMI: string (nullable = true)



# Overview

In [31]:
[x for x in [1,2,4]]+ [x for x in [4,5,6]]

[1, 2, 4, 4, 5, 6]

In [33]:
[ StructField(col, StringType(), True) for col in string_columns ] + [ StructField(col, IntegerType(), True) for col in columns if col not in string_columns]

[StructField('Workout_Type', StringType(), True),
 StructField('Gender', StringType(), True),
 StructField('Age', IntegerType(), True),
 StructField('Weight (kg)', IntegerType(), True),
 StructField('Height (m)', IntegerType(), True),
 StructField('Max_BPM', IntegerType(), True),
 StructField('Avg_BPM', IntegerType(), True),
 StructField('Resting_BPM', IntegerType(), True),
 StructField('Session_Duration (hours)', IntegerType(), True),
 StructField('Calories_Burned', IntegerType(), True),
 StructField('Fat_Percentage', IntegerType(), True),
 StructField('Water_Intake (liters)', IntegerType(), True),
 StructField('Workout_Frequency (days/week)', IntegerType(), True),
 StructField('Experience_Level', IntegerType(), True),
 StructField('BMI', IntegerType(), True)]

In [43]:
#Make into ints/strings
"""
columns = df.columns
string_columns = ['Workout_Type', 'Gender']
schema = StructType(
    [ StructField(col, StringType(), True) for col in string_columns ] + [ StructField(col, IntegerType(), True) for col in columns if col not in string_columns]
)
"""

"\ncolumns = df.columns\nstring_columns = ['Workout_Type', 'Gender']\nschema = StructType(\n    [ StructField(col, StringType(), True) for col in string_columns ] + [ StructField(col, IntegerType(), True) for col in columns if col not in string_columns]\n)\n"

Count NULLs. There are none.

In [13]:
df.select([count( when( isnan(c), c)).alias(c) for c in df.columns]).show()

+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+---+
|Age|Gender|Weight (kg)|Height (m)|Max_BPM|Avg_BPM|Resting_BPM|Session_Duration (hours)|Calories_Burned|Workout_Type|Fat_Percentage|Water_Intake (liters)|Workout_Frequency (days/week)|Experience_Level|BMI|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+---+
|  0|     0|          0|         0|      0|      0|          0|                       0|              0|           0|             0|                    0|                            0|               0|  0|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+----------------------

In [22]:
#Save table to catalog
df.write.saveAsTable("health_data")
#Read catalog 
print(spark.catalog.listTables())

[Table(name='health_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]


In [28]:
temp_df = spark.sql("select _c1, _c2 from health_data limit 3")

In [ ]:
# Count the number of nulls in each column of the dataframe
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [25]:
#df.select("Age", "Weight (kg)", "Height (m)").show()
#df.groupBy("Age").count().show()
df.filter(df["Age"]==47).show()

+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
|Age|Gender|Weight (kg)|Height (m)|Max_BPM|Avg_BPM|Resting_BPM|Session_Duration (hours)|Calories_Burned|Workout_Type|Fat_Percentage|Water_Intake (liters)|Workout_Frequency (days/week)|Experience_Level|  BMI|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
| 47|Female|       66.8|      1.75|    199|    146|         56|                    1.13|          742.0|    Strength|          32.8|                  2.5|                            3|               2|21.81|
| 47|Female|       58.5|      1.79|    195|    131|         71|                    1.42|          837.0|        Yoga|          30.3|                  2.4|              

As one can see, the columns are set arbitrarily.
Instead, we want to set the first row to the column names

In [30]:
temp_df.toPandas()

,_c1,_c2
0,Gender,Weight (kg)
1,Male,88.3
2,Female,74.9


24/11/19 02:18:27 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 911560 ms exceeds timeout 120000 ms
24/11/19 02:18:27 WARN SparkContext: Killing executors is not supported by current scheduler.
24/11/19 02:18:34 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc

In [13]:
row_1 = df.collect()[0]

In [14]:
row_1

Row(_c0='Age', _c1='Gender', _c2='Weight (kg)', _c3='Height (m)', _c4='Max_BPM', _c5='Avg_BPM', _c6='Resting_BPM', _c7='Session_Duration (hours)', _c8='Calories_Burned', _c9='Workout_Type', _c10='Fat_Percentage', _c11='Water_Intake (liters)', _c12='Workout_Frequency (days/week)', _c13='Experience_Level', _c14='BMI')